# 综合应用特征工程、分类器调参、K折交叉验证构建分类器

## 导入数据

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold    # 使用十折交叉运算进行训练集和测试集的拆分
from sklearn import svm     # 支持向量机

iris_data = load_iris()
iris_features = iris_data.data
iris_labels = iris_data.target

print('鸢尾花数据集特征集的特征向量前十条为：\n', iris_features[:10])
print('鸢尾花数据集特征集的标签前十条为：\n', iris_labels[:10])

鸢尾花数据集特征集的特征向量前十条为：
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
鸢尾花数据集特征集的标签前十条为：
 [0 0 0 0 0 0 0 0 0 0]


## 采用衍生变量的方法为数据集构建新特征

In [4]:
# 对花瓣长宽、萼片长宽分别进行加工，构建新的特征（新特征的实验在标准化前的数据集上进行）
sepal_area = np.array([iris_features[index][0] * iris_features[index][1] for index in range(len(iris_features))])
petal_area = np.array([iris_features[index][2] * iris_features[index][3] for index in range(len(iris_features))])

# 将新构建的矩阵进行转置，以和原数据在记录数的形状上相同，方便后续拼接
new_features = np.array([sepal_area, petal_area]).T

iris_features_added_new = np.column_stack((iris_features, new_features)) 

print('添加新特征后的鸢尾花数据集特征集的特征向量前十条为：\n', iris_features_added_new[:10])

添加新特征后的鸢尾花数据集特征集的特征向量前十条为：
 [[ 5.1   3.5   1.4   0.2  17.85  0.28]
 [ 4.9   3.    1.4   0.2  14.7   0.28]
 [ 4.7   3.2   1.3   0.2  15.04  0.26]
 [ 4.6   3.1   1.5   0.2  14.26  0.3 ]
 [ 5.    3.6   1.4   0.2  18.    0.28]
 [ 5.4   3.9   1.7   0.4  21.06  0.68]
 [ 4.6   3.4   1.4   0.3  15.64  0.42]
 [ 5.    3.4   1.5   0.2  17.    0.3 ]
 [ 4.4   2.9   1.4   0.2  12.76  0.28]
 [ 4.9   3.1   1.5   0.1  15.19  0.15]]


## 构建K折交叉检验模型

In [6]:
# 构建5折交叉检验模型
kf = KFold(n_splits = 5, shuffle = True)

test_score_basic_data = []
test_score_feature_engineered_data = []

counter = 1

# 分别对每一次K折交叉检验所提供的数据进行分类器构建及分类
for train_index, test_index in kf.split(iris_features):
    X_train_basic, X_test_basic, Y_train_basic, Y_test_basic = iris_features[train_index],\
                                                               iris_features[test_index],\
                                                               iris_labels[train_index],\
                                                               iris_labels[test_index]
    X_train_new, X_test_new, Y_train_new, Y_test_new = iris_features_added_new[train_index],\
                                                       iris_features_added_new[test_index],\
                                                       iris_labels[train_index],\
                                                       iris_labels[test_index]
    
    # 训练模型（未加入新特征）
    svm_classifier_basic = svm.SVC(C=1.0, kernel = 'rbf', decision_function_shape = 'ovr', gamma = 0.01)
    svm_classifier_basic.fit(X_train_basic, Y_train_basic)
    
    # 训练模型（加入新特征）
    svm_classifier_new = svm.SVC(C=1.0, kernel = 'rbf', decision_function_shape = 'ovr', gamma = 0.01)
    svm_classifier_new.fit(X_train_new, Y_train_new)
    
    print(str(counter), '本轮基础数据特征测试集得分为：', svm_classifier_basic.score(X_test_basic, Y_test_basic))
    test_score_basic_data.append(svm_classifier_basic.score(X_test_basic, Y_test_basic))
    
    print(str(counter), '本轮特征工程后的数据特征测试集得分为：', svm_classifier_new.score(X_test_new, Y_test_new))
    test_score_feature_engineered_data.append(svm_classifier_new.score(X_test_new, Y_test_new))
    
    counter += 1
    print("*" * 50)
    
print("基础数据特征下的K折交叉验证SVM测试集平均得分：", np.mean(test_score_basic_data))
print("特征工程后的K折交叉验证SVM测试集平均得分为：", np.mean(test_score_feature_engineered_data))

1 本轮基础数据特征测试集得分为： 0.9
1 本轮特征工程后的数据特征测试集得分为： 0.9333333333333333
**************************************************
2 本轮基础数据特征测试集得分为： 0.9333333333333333
2 本轮特征工程后的数据特征测试集得分为： 0.8666666666666667
**************************************************
3 本轮基础数据特征测试集得分为： 0.8666666666666667
3 本轮特征工程后的数据特征测试集得分为： 0.9666666666666667
**************************************************
4 本轮基础数据特征测试集得分为： 1.0
4 本轮特征工程后的数据特征测试集得分为： 1.0
**************************************************
5 本轮基础数据特征测试集得分为： 0.9666666666666667
5 本轮特征工程后的数据特征测试集得分为： 1.0
**************************************************
基础数据特征下的K折交叉验证SVM测试集平均得分： 0.9333333333333333
特征工程后的K折交叉验证SVM测试集平均得分为： 0.9533333333333334
